In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-125657")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126622
Azure region: southcentralus
Subscription id: a1ebbd25-46bc-446b-9547-5acab9b0125a
Resource group: aml-quickstarts-126622


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=30)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-17T13:19:47.728000+00:00', 'errors': None, 'creationTime': '2020-11-17T13:19:44.981989+00:00', 'modifiedTime': '2020-11-17T13:20:00.401321+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os

# Specify parameter sampler
#from azureml.train.hyperdrive import normal, uniform, choice
ps = RandomParameterSampling({
        '--C': choice(100, 10, 1.0, 0.1, 0.01),
        '--max_iter': choice(100, 110, 120)
       
    })
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy= BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)
#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =SKLearn("./", compute_target=compute_target, vm_size='STANDARD_D2_V2', 
             vm_priority='Dedicated', entry_script="train.py", 
             script_params=None, use_docker=True, custom_docker_image=None, 
             image_registry_details=None, user_managed=False, conda_packages=None, 
             pip_packages=None, conda_dependencies_file_path=None, pip_requirements_file_path=None, 
             conda_dependencies_file=None, pip_requirements_file=None, environment_variables=None, 
             environment_definition=None, inputs=None, shm_size=None, resume_from=None, 
             max_run_duration_seconds=None, framework_version=None, _enable_optimized_mode=False, 
             _disable_validation=True, _show_lint_warnings=False, _show_package_warnings=False)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
#run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print(parameter_values)

joblib.dump(value = best_run.id, filename= 'outputs/model.joblib')
best_run.register_model('best_model', model_path = 'outputs/model.joblib')
#best_run.download_file("model.pkl", "outputs/model.pkl")

Best Run Id:  HD_36ba9eb3-5e8e-45ad-b484-6914b5c33dd3_4

 Accuracy: 0.9121396054628225
['--C', '1', '--max_iter', '120']


Model(workspace=Workspace.create(name='quick-starts-ws-126622', subscription_id='a1ebbd25-46bc-446b-9547-5acab9b0125a', resource_group='aml-quickstarts-126622'), name=best_model, id=best_model:1, version=1, tags={}, properties={})

### Auto ML

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [8]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
from sklearn.model_selection import train_test_split
#create train test split
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=223)
#join the train_x and train_y to create train dataset
train_df=pd.concat([train_x,train_y], axis=1)

In [11]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(train_df).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)

 

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    debug_log = 'automl_errors.log',
    compute_target=compute_target,
    training_data=train_data,
    label_column_name=label,
    featurization='auto',
    n_cross_validations=5)

In [13]:
# Submit your automl run

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "automl-experiment")
local_run = experiment.submit(automl_config, show_output=False)

from azureml.widgets import RunDetails
RunDetails(local_run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [14]:
# Retrieve and save your best automl model.
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)
best_run.get_tags()
best_run.properties['model_name']


import joblib
joblib.dump(fitted_model, 'outputs/fitted_model.joblib')
best_run.register_model("auto-ml" , model_path = './outputs/')

Run(Experiment: automl-experiment,
Id: AutoML_6c0024ca-aaeb-4e0a-8944-b449fe8a21c6_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.01,
                                                                                                  min_samples_split=0.01,
                  

Model(workspace=Workspace.create(name='quick-starts-ws-126622', subscription_id='a1ebbd25-46bc-446b-9547-5acab9b0125a', resource_group='aml-quickstarts-126622'), name=auto-ml, id=auto-ml:1, version=1, tags={}, properties={})

In [15]:
# Optionally, delete the Azure Machine Learning Compute cluster
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

